In [ ]:
# Install OpenCV with CUDA support
!apt-get update
!apt-get install -y libopencv-dev libopencv-core-dev libopencv-imgproc-dev
!pip install opencv-python-headless


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [945 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,559 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [3,041 kB]
Get:13 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy In

In [ ]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
%%writefile non_local_means_denoising.cu
#include <opencv2/opencv.hpp>
#include <opencv2/core/cuda.hpp>
#include <iostream>
#include <vector>
#include <filesystem>
#include <chrono>
#include <string>
#include <cuda_runtime.h>
#include <device_launch_parameters.h>
#include <cmath>

namespace fs = std::filesystem;
using namespace cv;
using namespace std;

#define CUDA_CHECK(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true) {
    if (code != cudaSuccess) {
        fprintf(stderr, "GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
        if (abort) exit(code);
    }
}

void nonLocalMeansDenoisingGPU(const Mat& inputImage, Mat& outputImage, double h, double hForColorComponents, int templateWindowSize, int searchWindowSize) {
    fastNlMeansDenoisingColored(inputImage, outputImage, h, hForColorComponents, templateWindowSize, searchWindowSize);
}

double psnr(const Mat& I1, const Mat& I2) {
    CV_Assert(I1.size() == I2.size() && I1.type() == I2.type());

    Mat s1;
    absdiff(I1, I2, s1);
    s1.convertTo(s1, CV_32F);

    s1 = s1.mul(s1);
    Scalar s = sum(s1);
    double sse = s.val[0] + s.val[1] + s.val[2];

    if (sse <= 1e-10) return 0;

    double mse = sse / (I1.channels() * I1.total());
    double psnr = 10.0 * log10((255 * 255) / mse);
    return psnr;
}

bool endsWith(const string& str, const string& suffix) {
    return str.size() >= suffix.size() && str.compare(str.size() - suffix.size(), suffix.size(), suffix) == 0;
}

void processImage(const string& imagePath, const string& outputPath, double h, double hForColorComponents, int templateWindowSize, int searchWindowSize, double& totalInferenceTime, double& totalPSNR, int& imageCount) {
    Mat image = imread(imagePath, IMREAD_COLOR);
    if (image.empty()) {
        cerr << "Error loading image: " << imagePath << endl;
        return;
    }

    if (image.rows > 720 || image.cols > 1280) {
        cerr << "Image size exceeds the allowed dimensions of 1280x720p" << endl;
        return;
    }

    auto start = chrono::high_resolution_clock::now();
    Mat result;
    nonLocalMeansDenoisingGPU(image, result, h, hForColorComponents, templateWindowSize, searchWindowSize);
    auto end = chrono::high_resolution_clock::now();
    double inference_time = chrono::duration<double, milli>(end - start).count();

    imwrite(outputPath, result);

    double psnrValue = psnr(image, result);

    totalInferenceTime += inference_time;
    totalPSNR += psnrValue;
    ++imageCount;
}

void printGPUInfo() {
    cout << "GPU Metrics:" << endl;
    system("nvidia-smi --query-gpu=memory.used,utilization.gpu --format=csv,nounits,noheader | awk -F', ' '{print \"GPU Memory Usage (MB): \" $1 \" MB\"; print \"GPU Load (%): \" $2}'");
}

void processDataset(const string& rootFolder, double h, double hForColorComponents, int templateWindowSize, int searchWindowSize) {
    vector<string> noisyFolders = {"noisy5", "noisy15", "noisy35"};
    vector<string> denoisedFolders = {"denoised5", "denoised15", "denoised35"};

    double totalInferenceTime = 0;
    double totalPSNR = 0;
    int imageCount = 0;

    auto startTotalTime = chrono::high_resolution_clock::now();

    for (size_t i = 0; i < noisyFolders.size(); ++i) {
        string inputFolder = rootFolder + "/" + noisyFolders[i];
        string outputFolder = rootFolder + "/" + denoisedFolders[i];

        if (!fs::exists(outputFolder)) {
            fs::create_directory(outputFolder);
        }

        for (const auto& entry : fs::directory_iterator(inputFolder)) {
            string imagePath = entry.path().string();
            if (endsWith(imagePath, ".png") || endsWith(imagePath, ".jpg") || endsWith(imagePath, ".jpeg")) {
                string outputPath = outputFolder + "/" + entry.path().filename().string();
                processImage(imagePath, outputPath, h, hForColorComponents, templateWindowSize, searchWindowSize, totalInferenceTime, totalPSNR, imageCount);
            }
        }
    }

    auto endTotalTime = chrono::high_resolution_clock::now();
    double totalTimeTaken = chrono::duration<double, milli>(endTotalTime - startTotalTime).count();

    if (imageCount > 0) {
        double avgInferenceTime = totalInferenceTime / imageCount;
        double avgPSNR = totalPSNR / imageCount;

        cout << "Average Inference Time (ms): " << avgInferenceTime << endl;
        cout << "Average PSNR (dB): " << avgPSNR << endl;

        printGPUInfo();
    } else {
        cout << "No images processed." << endl;
    }
}

int main() {
    string rootFolder = "/content/drive/MyDrive/dataset";
    double h = 10.0;
    double hForColorComponents = 10.0;
    int templateWindowSize = 7;
    int searchWindowSize = 21;

    processDataset(rootFolder, h, hForColorComponents, templateWindowSize, searchWindowSize);

    return 0;
}


Writing non_local_means_denoising.cu


In [ ]:
!nvcc -diag-suppress=611 -o non_local_means_denoising non_local_means_denoising.cu `pkg-config --cflags --libs opencv4`
!./non_local_means_denoising


/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::buildMaps" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::warp" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(100): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is only partially overridden in class "cv::detail::FeatherBlender"
  class FeatherBlender : public Blender
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(127): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is